# Setup

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

2022-10-31 10:26:09.289560: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-31 10:26:09.293368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-31 10:26:09.293380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-31 10:26:10.580226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-31 10:26:10.580251: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

Turn on TFE debugging flags first thing

In [3]:
tfe.get_config().set_debug_mode(True)

# Computation

`multiply_three` function multiply input with scalar 3 and we can directly check multiply_x's input with python `print`

In [4]:
@tfe.function
def multiply_three(x):
    # only reveal secret value in debug
    print(x.reveal().to_native())
    return x * 3

x = tfe.define_private_input("input-provider", lambda: tf.random.uniform([2,2]))
multiply_three(x)

tf.Tensor(
[[0.14231873 0.25553894]
 [0.68479919 0.57910156]], shape=(2, 2), dtype=float64)


ABY3PrivateTensor(shape=(2, 2), share_type=ARITHMETIC)